In [1]:
import numpy as np
from PIL import Image
import pandas as pd
%config Completer.use_jedi = False

In [2]:
image_names = !ls ../raw_data/images_training_rev1/ #loading images_names.jpg to list
df_image_names = pd.DataFrame(image_names)  
df_image_names = df_image_names.rename(columns={0:'image'}) 
df_training_solutions_rev1 = pd.read_csv('../raw_data/training_solutions_rev1.csv')
#loading y dataframe

In [3]:
df_training_solutions_rev1['image'] = df_image_names['image']
df_X_y = df_training_solutions_rev1.drop(columns=['GalaxyID'])
df_X_y.shape

(61578, 38)

In [4]:
df_X_y_sample = df_X_y.sample(400)

In [5]:
df_X_y_train = df_X_y_sample[:200]
df_X_y_test = df_X_y_sample[200:]

In [6]:
import tensorflow as tf

def load_image(image):
    '''returns one image'''
    folder = f'../raw_data/images_training_rev1/{image}'
    img = Image.open(folder)
    img_array = np.array(img)
    return np.resize(img_array, (224,224,3))
    
def load_images(df):
    '''returns array of images'''
    img_list = []
    for _, row in df.iterrows():
        img = load_image(row["image"])
        img_list.append(img) 
    return np.stack(img_list)

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, df, batch_size=64, shuffle=True, ):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        '''returns number of minibatches per epoch'''
        return len(self.indices) // self.batch_size
    
    def on_epoch_end(self):
        '''shuffles the indices '''
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
            
    def __get_X_image(self, df):
        '''returns images'''
        X = load_images(df)
        return X 
             
    def __get_Y(self, df):
        '''returns y'''
        return np.array(df.drop(columns=["image"]))
    
    def _get_data(self, batch):
        '''returns batch of images and y'''
        df_batch = self.df.query("index in @batch") 
        X = self.__get_X_image(df_batch)
        y = self.__get_Y(df_batch)
        return X, y
    
    def __getitem__(self, index):
        '''creates batches and returns final X and y'''
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        X, y = self._get_data(batch)
        return X, y

In [7]:
datagenerator_train = DataGenerator(df=df_X_y_train, shuffle=True)
datagenerator_test = DataGenerator(df=df_X_y_test, shuffle=True)

In [8]:
import keras 
import tensorflow.keras as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Concatenate, Conv2D, Flatten
from keras import Sequential
from tensorflow.keras.models import Model

model = Sequential()
model.add(Input(shape=(224,224,3)))
model.add(Conv2D(22, 22, activation='relu'))
model.add(Conv2D(16, 16, activation='relu'))
model.add(Conv2D(2, 2, activation='relu'))
model.add(Flatten())
model.add(Dense(37, activation='linear'))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 203, 203, 22)      31966     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 188, 188, 16)      90128     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 187, 187, 2)       130       
_________________________________________________________________
flatten (Flatten)            (None, 69938)             0         
_________________________________________________________________
dense (Dense)                (None, 37)                2587743   
Total params: 2,709,967
Trainable params: 2,709,967
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='mse',  #loss a voir
              optimizer='adam',
              metrics='mse')

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=7, restore_best_weights=True )
model.fit(datagenerator_train, batch_size=32, epochs=500, callbacks=es, verbose=1, validation_data=datagenerator_test)

Epoch 1/500
3/3 [==============================] - 235s 69s/step - loss: 3.9940 - mse: 3.9940 - val_loss: 0.0717 - val_mse: 0.0717
Epoch 2/500
3/3 [==============================] - 174s 62s/step - loss: 0.0795 - mse: 0.0795 - val_loss: 0.0702 - val_mse: 0.0702
Epoch 3/500
3/3 [==============================] - 171s 61s/step - loss: 0.0796 - mse: 0.0796 - val_loss: 0.0696 - val_mse: 0.0696
Epoch 4/500
3/3 [==============================] - 171s 62s/step - loss: 0.0759 - mse: 0.0759 - val_loss: 0.0701 - val_mse: 0.0701
Epoch 5/500
3/3 [==============================] - 168s 61s/step - loss: 0.0768 - mse: 0.0768 - val_loss: 0.0689 - val_mse: 0.0689
Epoch 6/500
3/3 [==============================] - 200s 76s/step - loss: 0.0755 - mse: 0.0755 - val_loss: 0.0683 - val_mse: 0.0683
Epoch 7/500
3/3 [==============================] - 232s 84s/step - loss: 0.0756 - mse: 0.0756 - val_loss: 0.0686 - val_mse: 0.0686
Epoch 8/500
3/3 [==============================] - 243s 86s/step - loss: 0.0751 - m

3/3 [==============================] - 187s 67s/step - loss: 0.0508 - mse: 0.0508 - val_loss: 0.0457 - val_mse: 0.0457
Epoch 64/500
3/3 [==============================] - 186s 66s/step - loss: 0.0509 - mse: 0.0509 - val_loss: 0.0452 - val_mse: 0.0452
Epoch 65/500
3/3 [==============================] - 185s 66s/step - loss: 0.0502 - mse: 0.0502 - val_loss: 0.0445 - val_mse: 0.0445
Epoch 66/500
3/3 [==============================] - 185s 66s/step - loss: 0.0511 - mse: 0.0511 - val_loss: 0.0447 - val_mse: 0.0447
Epoch 67/500
3/3 [==============================] - 185s 66s/step - loss: 0.0489 - mse: 0.0489 - val_loss: 0.0443 - val_mse: 0.0443
Epoch 68/500
3/3 [==============================] - 183s 65s/step - loss: 0.0489 - mse: 0.0489 - val_loss: 0.0440 - val_mse: 0.0440
Epoch 69/500
3/3 [==============================] - 185s 66s/step - loss: 0.0498 - mse: 0.0498 - val_loss: 0.0444 - val_mse: 0.0444
Epoch 70/500
3/3 [==============================] - 184s 66s/step - loss: 0.0483 - mse: 0